In [1]:
#from gensim.models import Word2Vec
#from gensim.models.word2vec import LineSentence
import gensim
import gzip
import os
import glob
import csv
import multiprocessing
import numpy as np

word2vec_modell = 'wanmei'
Embsize = 150
stride = 1
Embepochs = 50
kmer_len3 = 3
kmer_len4 = 4
kmer_len5 = 5
kmer_len6 = 6

def Gen_Words(sequences,kmer_len,s):
		out=[]

		for i in sequences:

				kmer_list=[]
				for j in range(0,(len(i)-kmer_len)+1,s):

							kmer_list.append(i[j:j+kmer_len])

				out.append(kmer_list)

		return out


def train(sequences,kmer_len):
	print('training word2vec modell')
	document= Gen_Words(sequences,kmer_len,stride)
	#print(document)
	modell = gensim.models.Word2Vec (document, window=int(12 / stride), min_count=0, size=Embsize,workers=multiprocessing.cpu_count())
	modell.train(document,total_examples=len(document),epochs=Embepochs)
	modell.save(word2vec_modell+str(kmer_len))
	return document

def read_fasta_file():
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh = open('wanmei.txt', 'r')
    seq = []
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\n', '').replace('\r', ''))
    fh.close()
    matrix_data = np.array([list(e) for e in seq])
    #print(matrix_data)
    return seq

sequences = read_fasta_file()
document3=train(sequences,kmer_len3)
document4=train(sequences,kmer_len4)
document5=train(sequences,kmer_len5)
document6=train(sequences,kmer_len6)

model3 = gensim.models.Word2Vec.load(word2vec_modell+str(kmer_len3))
model4 = gensim.models.Word2Vec.load(word2vec_modell+str(kmer_len4))
model5 = gensim.models.Word2Vec.load(word2vec_modell+str(kmer_len5))
model6 = gensim.models.Word2Vec.load(word2vec_modell+str(kmer_len6))



training word2vec modell


In [2]:
#softmax: 在多分类中常用的激活函数，是基于逻辑回归的。
#softplus：softplus(x)=log(1+e^x)，近似生物神经激活函数，最近出现的。
#Relu：近似生物神经激活函数，最近出现的。
#tanh：双曲正切激活函数，也是很常用的。
#sigmoid：S型曲线激活函数，最常用的。
#hard_sigmoid：基于S型激活函数。
#linear：线性激活函数，最简单的

import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        pdf = PdfPages('x.pdf')

        plt.figure()

        plt.rcParams['figure.dpi'] = 350

     # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="center right")
        plt.title('Acc-loss curve of AD_RFHCP')
        pdf.savefig()
        plt.close()
        pdf.close()
        plt.show()
history = LossHistory()

Using TensorFlow backend.
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (type, 1) or '1type' as

In [3]:
X_train=[]
for i in range(0,17808):
    s=[]
#     for word in document3[i]:
#         s.append(model3.wv[word])
    
    for word in document3[i]:
        s.append(model3.wv[word])

#     for word in document5[i]:
#         s.append(model5.wv[word])
        
        
        
    X_train.append(s)
    
    
    
    
# print(np.shape(X_train))

# for word2 in document4:
#     s=[]
#     for i2 in word2:
#         s.append(model4.wv[i2])
#     X_train.append(s)
# print(np.shape(X_train))

# for word3 in document5:
#     s=[]
#     for i3 in word3:
#         s.append(model5.wv[i3])
#     X_train.append(s)
# print(np.shape(X_train))
    


In [4]:
np.shape(X_train)

(17808, 39, 150)

In [5]:
import numpy as around
X_train=np.around(X_train,4)


In [6]:
import pandas as pd
import numpy as np

#t_x=pd.read_csv('wanmeir.csv', header=None, index_col=None)
t_y=pd.read_csv('w_Y.csv', header=None, index_col=None)

xx= np.array(X_train)
xx_y=t_y.values


#xx = np.expand_dims(xx, axis=2)



#tt_x=pd.read_csv('fufufud.csv', header=None, index_col=None)
#tt_y=pd.read_csv('fuy.csv', header=None, index_col=None)

#test_x=tt_x.values
#test_y=tt_y.values


#test_x = np.expand_dims(test_x, axis=2)

In [7]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import math
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [9]:
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,GlobalAveragePooling1D
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten
from sklearn.metrics import roc_curve, auc,roc_auc_score 
from keras.layers import TimeDistributed ,advanced_activations

from sklearn.model_selection import train_test_split
import keras as K
from keras.layers import LSTM, Bidirectional
nb_filter=32
filter_length=8
def creat_model():
    global xx
    init = K.initializers.glorot_uniform()
    #scheduler = keras.callbacks.ReduceLROnPlateau(simple_adam, 'max', factor=0.5, patience=3)
    model = K.models.Sequential()


    #model.add(Convolution1D(nb_filter=64, filter_length=1, input_shape=(500, 10)))
    #model.add(Activation('relu'))
    #model.add(Flatten())
    #model.add(Dropout(0.4))
    # 定义卷积层

    # 编译模型
    model = keras.Sequential()
    acctivation=keras.layers.advanced_activations.LeakyReLU()  

    model.add(Convolution1D(nb_filter, filter_length, strides=1, padding='valid'))
#    model.add(keras.layers.MaxPool1D(pool_size=2, strides=2, padding="valid"))

    model.add(Dropout(0.1))

    model.add(Convolution1D(2*nb_filter, filter_length, strides=1, padding='valid'))
#    model.add(keras.layers.MaxPool1D(pool_size=2, strides=2, padding="valid"))

    model.add(Dropout(0.2))    
    model.add(Convolution1D(4*nb_filter, filter_length, strides=1, padding='valid'))
    model.add(keras.layers.MaxPool1D(pool_size=2, strides=2, padding="valid"))
#
    model.add(Dropout(0.3))  
#
#   model.add(Convolution1D(4*nb_filter, filter_length, strides=1, padding='valid'))
   # model.add(keras.layers.MaxPool1D(pool_size=2, strides=2, padding="valid"))
  #  model.add(Dropout(0.2))

 #   model.add(TimeDistributed(Dense(32)))
#    model.add(keras.layers.LSTM(32, return_sequences=True))
#    model.add(Bidirectional(LSTM(16, return_sequences=True)))

#    model.add(CRF(32))
    model.add(Flatten())

  #  model.add(K.layers.Dense(units=100 ,kernel_initializer=init, activation='relu'))
   # model.add(Dropout(0.2))
#    model.add(K.layers.Dense(units=100 ,kernel_initializer=init, activation='relu'))
   # model.add(Dropout(0.2))
    model.add(K.layers.Dense(units=1, kernel_initializer=init, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.001),metrics=['accuracy'])
    return model

from sklearn.model_selection import StratifiedKFold

def train():
    kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
    for train, test in kfold.split(xx, xx_y):
        train_x=xx[train]
        train_y=xx_y[train]
        test_x=xx[test]
        test_y=xx_y[test]
        train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size=0.8, test_size=0.2, random_state=0)
        print(train_x.shape,test_x.shape,val_x.shape)
        model = creat_model()
        max_epochs = 10
        print("Starting training ")
        h = model.fit(train_x, train_y, epochs=max_epochs, batch_size=128,shuffle=True, verbose=1,callbacks=[history],validation_data=(val_x, val_y))
        print("Training finished \n")

        eval = model.evaluate(test_x, test_y, verbose=1)
        print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (eval[0], eval[1] * 100) )
        print(model.summary())

def train2():
    train_x, test_x, train_y, test_y = train_test_split(xx, xx_y, train_size=0.8, test_size=0.2, random_state=0)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, train_size=0.8, test_size=0.2, random_state=0)
    #train_x, val_x, train_y, val_y = train_test_split(xx, xx_y, train_size=0.8, test_size=0.2, random_state=0)

    print(train_x.shape,test_x.shape,val_x.shape)
    model = creat_model()
    max_epochs =15
    
    from keras.callbacks import ReduceLROnPlateau
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, mode='auto',factor=0.1)
    print(reduce_lr)
    print("Starting training ")
    h = model.fit(train_x, train_y, epochs=max_epochs, batch_size=32,shuffle=True, verbose=1,callbacks=[reduce_lr,history],validation_data=(val_x, val_y))
    print("Training finished \n")

    scores = model.evaluate(test_x, test_y, verbose=1)
    print("Evaluation on test data: loss = %0.6f accuracy = %0.2f%% \n" % (scores[0],scores[1] * 100) )
    y_score = model.predict_proba(test_x)
    print ('调用函数auc：', roc_auc_score(test_y, y_score))
    print(model.summary())
#    pd.DataFrame(test_y).to_csv('yy.csv',header=None,index=False)
#    pd.DataFrame(y_score).to_csv('xx.csv',header=None,index=False)
    fpr, tpr, thresholds = roc_curve(test_y.ravel(),y_score.ravel())
    aucc = auc(fpr, tpr)
    print ('auc：', aucc)
#     plt.figure()

#     plt.rcParams['figure.dpi'] = 350
#     plt.plot(fpr, tpr, c = 'r', lw = 2, alpha = 0.7, label = 'AUC=%.3f' % aucc)
#     plt.plot((0, 1), (0, 1), c = '#808080', lw = 1, ls = '--', alpha = 0.7)
#     plt.xlim((-0.01, 1.02))
#     plt.ylim((-0.01, 1.02))
#     plt.xticks(np.arange(0, 1.1, 0.1))
#     plt.yticks(np.arange(0, 1.1, 0.1))
#     plt.xlabel('False Positive Rate', fontsize=13)
#     plt.ylabel('True Positive Rate', fontsize=13)
#     plt.grid(b=True, ls=':')
#     plt.legend(loc='lower right', fancybox=True, framealpha=0.8, fontsize=12)
#     plt.title('AUC', fontsize=17)
#     plt.show()
  
#    Z_K = model.predict(test_x)
#    pd.DataFrame(Z_K).to_csv(str(eval[1] * 100)+'zzz_d.csv',header=None,index=False)
    y_predict = model.predict(test_x)
    for i in range(len(y_predict)):
        if(y_predict[i]<0.5):
            y_predict[i]=0
        else:
            y_predict[i]=1
    print(y_predict)
    pd.DataFrame(y_predict).to_csv('y_predict.csv',header=None,index=False)

    ACC=metrics.accuracy_score(test_y,y_predict)
    MCC=metrics.matthews_corrcoef(test_y,y_predict)
    ROC_AUC_area=metrics.roc_auc_score(test_y,y_score)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(test_y, y_predict)

    ACC=round(ACC,4)
    MCC=round(MCC,4)
    ROC_AUC_area=round(ROC_AUC_area,4)
    SN=round(SN,4)
    SP=round(SP,4)
    print(ACC)
    print(MCC)
    print(ROC_AUC_area)
    print(SN)
    print(SP)
if __name__ == '__main__':
 #   train()
    train2()
    import winsound 

    duration = 1000  # millisecond
    freq = 440  # Hz
    winsound.Beep(freq, duration)

(11396, 39, 150) (3562, 39, 150) (2850, 39, 150)
Starting training 
Train on 11396 samples, validate on 2850 samples
Epoch 1/15
11396/11396 [==============================] - 9s 766us/step - loss: 0.4647 - accuracy: 0.8446 - val_loss: 0.3102 - val_accuracy: 0.8881
Epoch 2/15
11396/11396 [==============================] - 9s 778us/step - loss: 0.3285 - accuracy: 0.8816 - val_loss: 0.3251 - val_accuracy: 0.8632
Epoch 3/15
11396/11396 [==============================] - 9s 829us/step - loss: 0.3064 - accuracy: 0.8858 - val_loss: 0.3024 - val_accuracy: 0.8870
Epoch 4/15
11396/11396 [==============================] - 10s 881us/step - loss: 0.3130 - accuracy: 0.8829 - val_loss: 0.2998 - val_accuracy: 0.8895
Epoch 5/15
11396/11396 [==============================] - 10s 892us/step - loss: 0.3044 - accuracy: 0.8881 - val_loss: 0.3279 - val_accuracy: 0.8842
Epoch 6/15
11396/11396 [==============================] - 10s 906us/step - loss: 0.3087 - accuracy: 0.8838 - val_loss: 0.2915 - val_accuracy: